In [ ]:
%matplotlib inline
import pandas as pd
import geopandas as gpd
import os

In [ ]:
os.getcwd()

In [ ]:
basepath = 'C:\\Users\\Sarah\\Nexus365\\Tom Russell - ghana-oxford-infrastructure-adaptation-2020'

In [ ]:
ghana = gpd.read_file(os.path.join(basepath, 'incoming\\Ghana.shp'))
ghana.set_crs("EPSG:4326")

# Energy

In [ ]:
energypath = os.path.join(basepath,'incoming/energy')

## Energy - Transmission Lines

In [ ]:
etransm_path = os.path.join(energypath, 'Energy Transmission Network/ID 4 World Bank Catalog - C.Aderne/ghanafinal.geojson')
GHA_etransm = gpd.read_file(etransm_path)

In [ ]:
GHA_etransm = gpd.overlay(GHA_etransm, ghana, how='intersection')

In [ ]:
GHA_etransm=GHA_etransm[['voltage_kV','year','status','length_km','geometry']]

In [ ]:
GHA_etransm=GHA_etransm.reset_index()

In [ ]:
GHA_etransm=GHA_etransm.rename(columns = {
    'index':'ID', 'voltage_kV':'VOLTAGE_kV','year':'YEAR','status':'STATUS','length_km':'LENGTH_km'
})

In [ ]:
GHA_etransm.head()

In [ ]:
GHA_etransm.to_file(os.path.join(
basepath,'data','infrastructure','GHA_transmissionlines.shp'))

## Energy - Power Plants

There are two dataframes with powerplants, one from the World Bank Catalog and one from ECOVREX

### Dataframe Powerplants World Bank Catalog:

In [ ]:
powerWB_path = os.path.join(energypath, 'Power plants & stations/ID 1 - World Bank Catalog (WEPP2006)/GHA_PowerPlants.shp')
GHA_powerplWB = gpd.read_file(powerWB_path)

In [ ]:
GHA_powerplWB = GHA_powerplWB.set_crs("EPSG:4326")

In [ ]:
GHA_powerplWB = GHA_powerplWB[['PLANT','GEN_TYPE','STATUS','SUM_MW','geometry',]]

GHA_powerplWB=GHA_powerplWB.rename(columns = {
        'PLANT':'NAME', 'GEN_TYPE':'TYPE','SUM_MW':'CAPACITY_MW'
    })

In [ ]:
GHA_powerplWB.head()

### Dataframe from ECOVREX:

In [ ]:
powerECOV_path = os.path.join(energypath, 'Power plants & stations/ID67-ecreee_energy_generators/ecreee_energy_generators.shp')
GHA_powerplECOV = gpd.read_file(powerECOV_path)

In [ ]:
GHA_powerplECOV = GHA_powerplECOV.set_crs("EPSG:4326")

GHA_powerplECOV = gpd.overlay(ghana, GHA_powerplECOV, how='intersection') - ask Tom

In [ ]:
GHA_powerplECOV = GHA_powerplECOV[(GHA_powerplECOV.country == "Ghana")]

In [ ]:
GHA_powerplECOV = GHA_powerplECOV[['title','eg_type','current_st','current_ca','geometry']]
GHA_powerplECOV = GHA_powerplECOV.rename(columns = {'title':'NAME','eg_type':'TYPE','current_st':'STATUS','current_ca':'CAPACITY_MW'})

Deleted columns: nid, eg_type_id, initiative, ree_cat, Size, Longitude & Latitude

In [ ]:
GHA_powerplECOV.head()

### Concatenate both data frames

In [ ]:
GHA_powerpl_concat = pd.concat([GHA_powerplWB, GHA_powerplECOV], sort=False, keys=['PP_WB','PP_ECOV'])
GHA_powerpl_concat = GHA_powerpl_concat.reset_index()

In [ ]:
GHA_powerpl_concat

In [ ]:
GHA_powerpl_concat.to_file(os.path.join(
basepath,'data','infrastructure','GHA_powerplant.shp'))